# Imports

In [2]:
import json
import requests

from datetime import datetime, timedelta, timezone, time, date
from itertools import product
from functools import reduce
from os import path

import pandas as pd
import numpy as np

In [2]:
# import functions from eclipses notebook
%run desktop/opentransit-metrics/mykelu/eclipses.py

# Get Stops

In [16]:
def get_stops(dates, routes, directions = [], new_stops = [], times = ("00:00", "23:59")):
    """
    get_stops
    
    Description:
        Returns every instance of a bus stopping at a given set of stops, on a given set of routes, during a given time period.

    Parameters:
        dates: an array of dates, formatted as strings in the form YYYY-MM-DD
        routes: an array of routes, each represented as a string
        directions: an array of strings representing the directions to filter
        stops: an array of strings representing the stops to filter
        times: a tuple with the start and end times (in UTC -8:00) as strings in the form HH:MM 

    Returns:
        stops: a DataFrame, filtered by the given directions and stops, with the following columns:
            VID: the vehicle ID
            Time: a datetime object representing the date/time of the stop
            Route: the route on which the stop occurred
            Stop: the stop at which the stop occurred
            Dir: the direction in which the stop occurred
    """
    bus_stops = pd.DataFrame(columns = ["VID", "DATE", "TIME", "SID", "DID", "ROUTE"])

    for route in routes:
        stop_ids = [stop['id']
            for stop
            in requests.get(f"http://restbus.info/api/agencies/sf-muni/routes/{route}").json()['stops']]

        for stop_id in stop_ids:
            # check if stops to filter were provided, or if the stop_id is in the list of filtered stops
            if (stop_id in new_stops) ^ (len(new_stops) == 0):
                for date in dates:
                    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: starting processing on stop {stop_id} on route {route} on {date}.")
                    start_time = int(datetime.strptime(f"{date} {timespan[0]} -0800", "%Y-%m-%d %H:%M %z").timestamp())*1000
                    end_time   = int(datetime.strptime(f"{date} {timespan[1]} -0800", "%Y-%m-%d %H:%M %z").timestamp())*1000

                    data = query_graphql(start_time, end_time, route)
#                     print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: performed query.")
                          
                    if data is None:  # API might refuse to cooperate
                        print("API probably timed out")
                        continue
                    elif len(data) == 0:  # some days somehow have no data
                        print(f"no data for {month}/{day}")
                        continue
                    else:
                        try:
                            stops = produce_stops(data, route)
                        except: # account for...bad requests?
                            print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: could not produce stops df for {stop_id} on route {route} on {date}. Skipping.")
                            continue
#                         print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: produced stops.")
                              
                        buses = produce_buses(data)
#                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: produced buses.")

                        stop = stops[stops['SID'] == stop_id].squeeze()
#                        print(stops['SID'].unique())
#                        print(buses)
                        try: 
                            buses = buses[buses['DID'] == stop['DID']]
                        except ValueError: # accounts for stops with no associated direction
                            print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: no direction associated to {stop_id} on route {route} on {date}. Skipping.")
                            continue
#                        print(buses)

                        eclipses = find_eclipses(buses, stop)
#                         print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: found eclipses.")
                              
                        nadirs = find_nadirs(eclipses)
#                         print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: found nadirs.")
                            
                        nadirs["TIME"] = nadirs["TIME"].apply(lambda x: datetime.fromtimestamp(x//1000, timezone(timedelta(hours = -8))))
                        nadirs['DATE'] = nadirs['TIME'].apply(lambda x: x.date())
                        nadirs['TIME'] = nadirs['TIME'].apply(lambda x: x.time())
                        nadirs["SID"] = stop_id
                        nadirs["DID"] = stop["DID"]
                        nadirs["ROUTE"] = route
                        old_length = len(bus_stops)
                        bus_stops = bus_stops.append(nadirs, sort = True)
#                         print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: finished processing.")

    # filter for directions
    if len(directions) > 0:
        bus_stops = bus_stops.loc[bus_stops['DID'].apply(lambda x: x in directions)]
     
    # prepare timestamp data; conditional accounts for an empty df
    if len(bus_stops) > 0:
        bus_stops['timestamp'] = bus_stops[['DATE', 'TIME']].apply(lambda x: datetime.strptime(f"{x['DATE'].isoformat()} {x['TIME'].isoformat()} -0800", 
                                                                                       "%Y-%m-%d %H:%M:%S %z"), axis = 'columns')
    
    return bus_stops

In [4]:
# get some stops
route = ["12", "14"]

timespan = ("08:00",
            "11:00")

dates = [
    "2018-11-12",
    "2018-11-13",
    "2018-11-14",
]

In [17]:
new_stops = get_stops(dates, route)

Wed Feb 06 12:44:21 AM: starting processing on stop 7941 on route 12 on 2018-11-12.
Wed Feb 06 12:44:39 AM: starting processing on stop 7941 on route 12 on 2018-11-13.
Wed Feb 06 12:44:48 AM: starting processing on stop 7941 on route 12 on 2018-11-14.
Wed Feb 06 12:44:59 AM: starting processing on stop 5859 on route 12 on 2018-11-12.
Wed Feb 06 12:45:10 AM: starting processing on stop 5859 on route 12 on 2018-11-13.
Wed Feb 06 12:45:18 AM: starting processing on stop 5859 on route 12 on 2018-11-14.
Wed Feb 06 12:45:29 AM: starting processing on stop 5851 on route 12 on 2018-11-12.
Wed Feb 06 12:45:39 AM: starting processing on stop 5851 on route 12 on 2018-11-13.
Wed Feb 06 12:45:47 AM: starting processing on stop 5851 on route 12 on 2018-11-14.
Wed Feb 06 12:45:58 AM: starting processing on stop 5844 on route 12 on 2018-11-12.
Wed Feb 06 12:46:08 AM: starting processing on stop 5844 on route 12 on 2018-11-13.
Wed Feb 06 12:46:16 AM: starting processing on stop 5844 on route 12 on 2018

In [13]:
#stop_filter = ['3477']

In [14]:
#new_stops = get_stops(dates, route, new_stops = stop_filter)

Wed Feb 06 12:20:06 AM: starting processing on stop 3477 on route 12 on 2018-11-12.
Wed Feb 06 12:20:32 AM: starting processing on stop 3477 on route 12 on 2018-11-13.
Wed Feb 06 12:20:42 AM: starting processing on stop 3477 on route 12 on 2018-11-14.


In [19]:
# TODO: parse direction as inbound/outbound? (remove route indicator)
# test directions (find unique directions) - possibly parse directions
new_stops.head()

,DATE,DID,ROUTE,SID,TIME,VID,timestamp
1,2018-11-12,12___O_F00,12,7941,08:00:13,8776,2018-11-12 08:00:13-08:00
4829,2018-11-12,12___O_F00,12,7941,10:16:05,8776,2018-11-12 10:16:05-08:00
7,2018-11-12,12___O_F00,12,7941,08:00:13,8705,2018-11-12 08:00:13-08:00
4268,2018-11-12,12___O_F00,12,7941,10:00:49,8705,2018-11-12 10:00:49-08:00
410,2018-11-12,12___O_F00,12,7941,08:11:44,8912,2018-11-12 08:11:44-08:00


In [20]:
len(new_stops)

8007

# Prepare Timestamp Data

In [ ]:
# # TODO: when to use date/time columns vs timestamps?
# new_stops['timestamp'] = new_stops[['DATE', 'TIME']].apply(lambda x: datetime.strptime(f"{x['DATE'].isoformat()} {x['TIME'].isoformat()} -0800", 
#                                                                                        "%Y-%m-%d %H:%M:%S %z"),  
#                                                            axis = 'columns')

In [ ]:
new_stops['timestamp'].head()

# Compute Average Waiting Time

In [25]:
# initial, slower implementation of get_wait_times

# find the smallest nonnegative waiting time
def absmin(series):
    return series[series >= 0].min()

# # input: df with entries from one day
# # possible optimzation: sort df by timestamp, then pick first timestamp > minute for each minute (need to time to make sure but should be faster)
def minimum_waiting_times(df, start_time, end_time, group):
    minute_range = [start_time + timedelta(minutes = i) for i in range((end_time - start_time).seconds//60)]
    wait_times = pd.DataFrame(columns = [])
    
    for minute in minute_range:
        df['WAIT'] = df['timestamp'].apply(lambda x: (x - minute).total_seconds())
        pivot = df[group + ['WAIT']].pivot_table(values = ['WAIT'], index = group, aggfunc = absmin)
        pivot['TIME'] = minute
        pivot = pivot.reset_index()
        wait_times = wait_times.append(pivot, sort = True)
        
    return wait_times

def all_wait_times(df, timespan, group, aggfuncs):
    dates = df['DATE'].unique()
    avg_over_pd = pd.DataFrame(columns = group + ['DATE', 'TIME', 'WAIT'])
    
    for date in new_stops['DATE'].unique():
        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: start processing {date}.")
        start_time = datetime.strptime(f"{date.isoformat()} {timespan[0]} -0800", "%Y-%m-%d %H:%M %z")
        end_time   = datetime.strptime(f"{date.isoformat()} {timespan[1]} -0800", "%Y-%m-%d %H:%M %z")
        daily_wait = minimum_waiting_times(new_stops[new_stops['DATE'] == date], start_time, end_time, group)
        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: found stops for {date}.")      
        #daily_wait = daily_wait.pivot_table(values = ['WAIT'], index = group).reset_index()
        daily_wait['DATE'] = date
        daily_wait['TIME'] = daily_wait['TIME'].apply(lambda x: x.time())
        avg_over_pd = avg_over_pd.append(daily_wait, sort = True)
    
    return avg_over_pd#.pivot_table(values = ['WAIT'], index = group, aggfunc = aggfuncs)

In [26]:
wait_times = all_wait_times(new_stops, timespan, ['SID'], {'WAIT': [np.mean]})

Wed Feb 06 02:51:56 AM: start processing 2018-11-12.


C:\Users\jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Wed Feb 06 02:52:45 AM: found stops for 2018-11-12.
Wed Feb 06 02:52:45 AM: start processing 2018-11-13.
Wed Feb 06 02:53:43 AM: found stops for 2018-11-13.
Wed Feb 06 02:53:43 AM: start processing 2018-11-14.
Wed Feb 06 02:54:34 AM: found stops for 2018-11-14.


In [28]:
# a faster implementation of get_wait_times
def get_wait_times(df, dates, timespan, group):
    """
    get_wait_times
    
    Description:
        Takes a DataFrame containing stops for a given route/timespan and returns the corresponding waiting times in that timespan.
        
    Parameters:
        df: a DataFrame containing stop times for a given route/timespan/date interval.
        dates: the range of dates over which to retrieve wait times.
        timespan: the timespan to compute wait times for.
        group: the columns to group over. Needed for sorting.
    
    Returns:
        wait_times: a DataFrame containing the waiting times over the given parameters.
    """
    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: starting!")
    
    # sort the DataFrame by time first
    df = df.sort_values(['timestamp']).reset_index()
    wait_times = pd.DataFrame(columns = [])      
    filters = product(*[new_stops[s].unique() for s in group])
    filters = [{group[i]:filter[i] for i in range(len(group))} for filter in filters]
          
    # include day range, take from dates       
    for filter in filters:
        filtered_waits = pd.DataFrame(columns = [])
        filtered_stops = df.loc[reduce((lambda x, y: x & y), [df.apply(lambda x: x[key] == filter[key], axis = 1) for key in filter.keys()]), :]
        start_time = datetime.strptime(f"{filter['DATE'].isoformat()} {timespan[0]} -0800", "%Y-%m-%d %H:%M %z")
        end_time   = datetime.strptime(f"{filter['DATE'].isoformat()} {timespan[1]} -0800", "%Y-%m-%d %H:%M %z")
        minute_range = [start_time + timedelta(minutes = i) for i in range((end_time - start_time).seconds//60)]
        current_index = 0

        for minute in minute_range:
            while current_index < len(filtered_stops):
                if filtered_stops.iloc[current_index]['timestamp'] >= minute:
                    break
                else:
                    current_index += 1
                                       
            # catches the case where current_stops = len(filtered_stops)
            try:
                filtered_waits = filtered_waits.append(filtered_stops.iloc[current_index])
            except:
                if len(filtered_stops) > 0:
                    filtered_waits = filtered_waits.append(filtered_stops.iloc[-1])

        filtered_waits.index = range(len(filtered_waits))
        filtered_waits['MINUTE'] = pd.Series(minute_range)
        wait_times = wait_times.append(filtered_waits)
        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: finished {filter}!")

    wait_times['WAIT'] = wait_times.apply(lambda x: (x['timestamp'] - x['MINUTE']).total_seconds(), axis = 'columns')
    wait_times.index = range(len(wait_times))
    wait_times['MINUTE'] = wait_times['MINUTE'].apply(lambda x: x.time())
    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: finishing!")
    return wait_times[['MINUTE', 'WAIT'] + group]

In [29]:
waits = get_wait_times(new_stops, dates, timespan, ['DATE', 'SID'])

Wed Feb 06 02:58:27 AM: starting!
Wed Feb 06 02:58:32 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '7941'}!
Wed Feb 06 02:58:35 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '5859'}!
Wed Feb 06 02:58:38 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '5851'}!
Wed Feb 06 02:58:42 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '5844'}!
Wed Feb 06 02:58:45 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '5839'}!
Wed Feb 06 02:58:50 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '5846'}!
Wed Feb 06 02:58:54 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '5841'}!
Wed Feb 06 02:58:58 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '5857'}!
Wed Feb 06 02:59:02 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '5848'}!
Wed Feb 06 02:59:07 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '5853'}!
Wed Feb 06 02:59:12 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '3087'}!
Wed Feb 0

C:\Users\jose\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Wed Feb 06 03:01:09 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '6881'}!
Wed Feb 06 03:01:13 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '6878'}!
Wed Feb 06 03:01:17 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '3486'}!
Wed Feb 06 03:01:21 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '3477'}!
Wed Feb 06 03:01:24 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '33476'}!
Wed Feb 06 03:01:28 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '3476'}!
Wed Feb 06 03:01:32 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '6877'}!
Wed Feb 06 03:01:35 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '6879'}!
Wed Feb 06 03:01:40 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '7592'}!
Wed Feb 06 03:01:44 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '7551'}!
Wed Feb 06 03:01:48 AM: finished {'DATE': datetime.date(2018, 11, 12), 'SID': '7552'}!
Wed Feb 06 03:01:52 AM: finished {'DATE': 

In [51]:
# TODO:
# serialization
# find route combinations to test (w/overlapping stops/directions)
# write other metrics

In [33]:
len(waits)

96660

In [34]:
len(wait_times)

94306

In [ ]:
# what should be done to instances of negative wait times?
# need to collect data until a stop has been found for all times in timespan
# don't know how to get this to work w/current graphql query
# => need to rewrite get_stops so that all times have a nonnegative wait?
# ez solution: 
waits[waits.apply(lambda x: x['WAIT'] < 0, axis = 'columns')]

# Export to csv

In [60]:
len(new_stops)

8007

In [66]:
def write_file(df, filename):
    with open(filename, 'w', newline = '') as csvfile:
        df.to_csv(csvfile, index = False)
        
    print(f"{filename} written!")

In [70]:
write_file(new_stops, './Desktop/opentransit-metrics/mykelu/stops.csv')

./Desktop/opentransit-metrics/mykelu/stops.csv written!


In [71]:
write_file(wait_times, './Desktop/opentransit-metrics/mykelu/wait_times.csv')

./Desktop/opentransit-metrics/mykelu/wait_times.csv written!


# Get Summary Statistics

In [ ]:
# summary stats over all waits
waits.describe()

In [36]:
def quantiles(series):
    return [np.percentile(series, i) for i in [5, 25, 50, 75, 95]]

def get_summary_statistics(df, group):
    waits = df.pivot_table(values = ['WAIT'], index = group, aggfunc = {'WAIT': [np.mean, np.std, quantiles]}).reset_index()
    waits.columns = ['_'.join(col) if col[0] == 'WAIT' else ''.join(col) for col in waits.columns.values]
    waits[[f"{i}th percentile" for i in [5, 25, 50, 75, 95]]] = waits['WAIT_quantiles'].apply(lambda x: pd.Series(x))
    waits = waits.drop('WAIT_quantiles', axis = 1)
    return waits

In [37]:
get_summary_statistics(waits, ['DATE', 'SID'])

,DATE,SID,WAIT_mean,WAIT_std,5th percentile,25th percentile,50th percentile,75th percentile,95th percentile
0,2018-11-12,3011,496.211111,355.583492,50.95,229.75,441.0,710.25,1145.00
1,2018-11-12,3013,482.094444,347.633158,33.15,192.25,426.5,668.75,1096.40
2,2018-11-12,3082,419.527778,285.577831,4.90,184.50,421.0,638.50,892.55
3,2018-11-12,3084,444.816667,293.290968,16.95,196.75,425.5,660.50,917.60
4,2018-11-12,3087,421.766667,292.228676,5.80,185.00,419.0,650.00,890.10
5,2018-11-12,3238,500.655556,345.209067,48.85,225.75,467.5,709.50,1101.00
6,2018-11-12,33476,539.255556,407.076457,13.50,184.00,471.5,847.50,1251.65
7,2018-11-12,3476,635.316667,455.744482,45.20,259.75,561.0,958.00,1460.35
8,2018-11-12,3477,586.683333,399.612151,50.90,254.50,531.0,845.50,1339.60
9,2018-11-12,3486,515.727778,344.694632,44.95,238.75,481.0,734.00,1124.30


In [38]:
new_stops.head()

,DATE,DID,ROUTE,SID,TIME,VID,timestamp
1,2018-11-12,12___O_F00,12,7941,08:00:13,8776,2018-11-12 08:00:13-08:00
4829,2018-11-12,12___O_F00,12,7941,10:16:05,8776,2018-11-12 10:16:05-08:00
7,2018-11-12,12___O_F00,12,7941,08:00:13,8705,2018-11-12 08:00:13-08:00
4268,2018-11-12,12___O_F00,12,7941,10:00:49,8705,2018-11-12 10:00:49-08:00
410,2018-11-12,12___O_F00,12,7941,08:11:44,8912,2018-11-12 08:11:44-08:00


In [39]:
# sort stops
new_stops = new_stops.sort_values(['timestamp'])

In [40]:
# aggregation function to find time length for trips
def tripdelta(series):
    return series['TIME'].max() - series['TIME'].min()

In [41]:
new_stops['TIME'] = new_stops.apply(lambda x: datetime.combine(x['DATE'], x['TIME']), axis = 'columns')

In [42]:
new_stops['TIME'] = new_stops['TIME'].apply(lambda x: x.timestamp())

In [43]:
new_stops[new_stops['SID'].apply(lambda x: x in ['5851', '5844'])].pivot_table(values = ['TIME'], index = ['VID'], aggfunc = tripdelta)

,TIME
VID,
8702,8572.0
8705,89982.0
8707,61.0
8708,45.0
8714,60.0
8722,60.0
8723,60.0
8728,75.0
8731,171121.0


In [50]:
new_stops[(new_stops['VID'] == '8702') & (new_stops['SID'].apply(lambda x: x in ['36498', '5844']))]

,DATE,DID,ROUTE,SID,TIME,VID,timestamp
6,2018-11-14,12___O_F00,12,5844,1.542182e+09,8702,2018-11-14 08:00:01-08:00
4478,2018-11-14,12___O_F00,12,5844,1.542191e+09,8702,2018-11-14 10:22:53-08:00


In [ ]:
stop_query = f"""{{
        trynState(agency: "muni",
                  startTime: "1510554087312",
                  endTime: "1510555087312",
                  routes: ["12"]) {{
            routes {{
                stops {{
                    sid
                    name
                    lat
                    lon
                }}
            }}
        }}
    }}
    """
query_url = f"https://06o8rkohub.execute-api.us-west-2.amazonaws.com/dev/graphql?query={stop_query}"

In [ ]:
now = datetime.now()
request = requests.get(query_url).json()['data']['trynState']['routes'][0]['stops']
print(datetime.now() - now)

In [ ]:
request['data']['trynState']['routes'][0]['stops']

In [3]:
request = requests.get(f"http://restbus.info/api/agencies/sf-muni/routes/14")

In [11]:
request.json().keys()

dict_keys(['id', 'title', 'shortTitle', 'color', 'textColor', 'bounds', 'stops', 'directions', 'paths', '_links'])

In [4]:
request.json()['stops']

[{'id': '6498',
  'code': '16498',
  'title': 'Steuart & Mission St',
  'lat': 37.7932499,
  'lon': -122.39328},
 {'id': '5623',
  'code': '15623',
  'title': 'Mission St & Spear St OB',
  'lat': 37.7923899,
  'lon': -122.39435},
 {'id': '5579',
  'code': '15579',
  'title': 'Mission St & Beale St',
  'lat': 37.79114,
  'lon': -122.3959199},
 {'id': '5528',
  'code': '15528',
  'title': 'Mission St & 1st St',
  'lat': 37.78992,
  'lon': -122.3974899},
 {'id': '5529',
  'code': '15529',
  'title': 'Mission St & 2nd St',
  'lat': 37.7877499,
  'lon': -122.40024},
 {'id': '5532',
  'code': '15532',
  'title': 'Mission St & 3rd St',
  'lat': 37.7859699,
  'lon': -122.40249},
 {'id': '5534',
  'code': '15534',
  'title': 'Mission St & 4th St',
  'lat': 37.7843599,
  'lon': -122.40452},
 {'id': '5536',
  'code': '15536',
  'title': 'Mission St & 5th St',
  'lat': 37.7827299,
  'lon': -122.40664},
 {'id': '5538',
  'code': '15538',
  'title': 'Mission St & 6th St',
  'lat': 37.78075,
  'lon':